The following code is an implementation of the Value class from the tutorial, based on memory and understanding.

In [1]:
# introduction

a = 2.0
b = -3.5
c = 9.0
d = 1.5
e =  a + b
f = c*e
L = f + d

# Essentially what backproppagation does is 
# evaluating the derivative of L with respect to
# its constituent variables, i.e. a, b, c, d, e, f

To start, a Value class has to be created to store floats, and to allow extra operations on the specific value

In [2]:
class Value:
    def __init__(self, data, _children=(), _op=""):
        self.data = data
        # stores its operands
        self._children = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        return Value(self.data + other.data, (self, other), "+")
    
    def __mul__(self, other):
        return Value(self.data * other.data, (self, other), "*")
    


In [3]:
a = Value(2.0)
b = Value(-3.0)
c = Value(5.0)
d = a*b + c
d

Value(data=-1.0)

In [4]:
# Visualisation of tree
from graphviz import Digraph

# Obtain nodes and edges from root node
def trace(root):
    nodes, edges = set(), set()
    def dfs(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._children:
                edges.add((child, v))
                dfs(child)
    dfs(root)
    return nodes, edges

def draw(root):
    dot = Digraph(format="svg", graph_attr={"rankdir": "LR"}) # Left to Right graph

    nodes, edges = trace(root)
    for n in nodes:
        # create unique uid for node
        uid = str(id(n))
        # Create node in graph
        dot.node(name = uid, label = f"{n.data:.4f}", shape="record")
        if n._op:
            # create node of operation if its a result of one
            dot.node(name = uid + n._op, label = n._op)
            # connect operation to its result node
            dot.edge(uid + n._op, uid)

    for n1, n2 in edges:
        # connect all nodes
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
        
    return dot


In [5]:
draw(d)

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH